# CS207 Project Group 11 - "The Differentiators"
# Milestone 2

*****

## Introduction

Derivatives are ubiquitous in many fields such as engineering design optimization, fluid dynamics and machine learning.
There are in general three ways to calculate the derivatives: automatic differentiation, numeric differentiation, and
symbolic differentiation. Automatic Differentiation (AD) brings a family of techniques that can calculate the partial
derivatives of any function at any point efficiently and accurately. Unlike numeric differentiation, AD does not have
the problem of floating point precision errors, since it calculates the derivative of a simple function, and keeps track of
these derivatives, and there is no need of step sizes. Compared to symbolic differentiation, AD is not as memory intense,
and can be much faster in terms of the calculation. Therefore, AD is an important way to calculate derivatives in
practice.

The software that we design calculates the derivatives given the user’s input using the forward mode of automatic differentiation,
and provides the user with an easy way to solve their optimization problem using derivatives.


## Background

At the core of Automatic Differentiation is the principle that functions implemented as computer code can be broken down into elementary functions, ranging from arithmetic operations (e.g. addition, subtraction etc.) and other functions (e.g. power, exponential, sin etc.). Hence, any differentiable function can be interpreted as a composition of different functions. 

For example, given a function, $f = sin^2(2x)$, it can be rewritten as:

$$ f = \phi_1(\phi_2(\phi_3(x))) $$ 

where $$ \phi_1(z) = z^2,   \phi_2(y) = sin(y) \text{ and } \phi_3(x) = 2x$$


In the forward mode, the chain rule can then be applied successively to each elementary component function to obtain the derivative of the function. Using the same example above, let $c$ be a real number:
$$ f'(c) =  \phi_3'(\phi_2(\phi_1(c))) \cdot \phi_2'(\phi_1(c)) \cdot \phi_1'(c)$$

Based on the example above, the derivative, $f'(c)$, can be evaluated based on the following function-derivative pairs at each stage of computing the function:

$$(\phi_1(c), \phi_1'(c))$$

$$(\phi_2(\phi_1(c)), (\phi_2'(\phi_1(c)) \cdot \phi_1'(c)))$$

$$(\phi_3(\phi_2(\phi_1(c))), \phi_3'(\phi_2(\phi_1(c)) \cdot \phi_2'(\phi_1(c)) \cdot \phi_1'(c))$$

Effectively, the forward mode computes the Jacobian-vector product, $Jp$. This decomposition can be represented via a computational graph structure of calculations, requiring initial values to be set for $x_1$, and $x'_1$:

$$x_1 \rightarrow^{\phi_3(x)} x_2 \rightarrow^{\phi_2(x)} x_3 \rightarrow^{\phi_1(x)} y $$

where $$ \phi_1(x) = x^2,   \phi_2(x) = sin(x) \text{ and } \phi_3(x) = 2x$$

At each stage of the function, the derivative of the function with respect to its argument is calculated. The exact values of the function and its derivative are used for the following function-derivative pair of values. An example of the computational trace for the equation $f = sin^2(2x)$ would look like this, for $x = \dfrac{\pi}{6}$. 

| Trace    | Elementary Operation &nbsp;&nbsp;&nbsp;| Derivative &nbsp;&nbsp;&nbsp; | $\left(f\left(a\right),  f^{\prime}\left(a\right)\right)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| :------: | :----------------------:               | :------------------------------: | :------------------------------: |
| $x_{1}$  | $\dfrac{\pi}{6}$                       | $1$                | $\left(\dfrac{\pi}{6}, 1\right)$ |
| $x_{2}$  | $2x_{1}$                               | $2\dot{x}_{1}$     | $\left(\dfrac{\pi}{3}, 2\right)$ |
| $x_{3}$  | $\sin(x_{2})$               | $\cos\left(x_{2}\right)\dot{x}_{2}$            | $\left(\dfrac{\sqrt{3}}{2}, 1\right)$ |
| $x_{4}$  | $x_{3}^{2}$                            | $2x_{3}\dot{x}_{3}$                   | $\left(\dfrac{3}{4}, \sqrt{3}\right)$ |

By evaluating the derivative at each step of the chain rule, we eventually obtain the value of the derivative $f'(x) = \sqrt{3}$ at $x = \dfrac{\pi}{6}$, as second entry of the final tuple in the table.

While the above illustrates the forward mode of AD (the focus of our package), AD also has a reverse mode. Without using chain rule, it first does a forward pass to store the partial derivatives, before undertaking a reverse pass, which starts from the final function to be differentiated, $y$. After fixing the derivative of the final function, it then computes the derivative of each component function with respect to its parent function recursively (using chain rule) until the derivative of the function with respect to the basic-level argument (e.g. $x_1$) can be calculated. 

In terms of efficiency, the forward mode is more efficient when the number of functions to evaluate is much greater than the number of inputs, whereas the reverse mode, which computes the Jacobian-transpose-vector-product is more efficient when the number of inputs is much greater than the number of functions.

## How to use `AutoDiff`

At this milestone, the recommended method to access the package is to download or clone the package's Github repo (https://github.com/the-differentiators/cs207-FinalProject.git). The user must ensure that the package's requirements (`numpy`) are installed, or install them manually.

By the final submission, the package will be available on PyPI and the user will be able to install the package in the standard way using `pip`. We will also provide a `yml` file which will be used by the user to create the appropriate environment and ensure that dependencies like `numpy` have been installed. Then, to use the package, the user will only need to import our package which will implicitly import any other packages used by `AutoDiff`.

The following steps will walk the user through a demo of how to install and use the `AutoDiff` package:

#### Importing `AutoDiff` and requirements
The following code is first used to change the path directory of this document to the same folder as that of the package   

In [1]:
# Code to change directory to src folder
import os
path = os.getcwd().replace('docs','src')
os.chdir(path)

Once the package's Github repo has been downloaded, the `AutoDiff` package can be imported with the following code by a Python file, with its working directory configured to the same directory as `AutoDiff.py`:

In [2]:
import AutoDiff as AD

For the purposes of this demo, we will import `numpy`, which is a requirement for the *AutoDiff* package, as well as the `Ad_Var` class from the `AutoDiff` package

In [3]:
import numpy as np
from AutoDiff import Ad_Var

#### Using `AutoDiff` to compute derivative of a scalar function

Below, we have included a basic demo for a scalar function, given a single input. The function used in the demo is  $f = sin^2(2x)$, which was used for illustration in the *Background* section earlier. Our objective is to use the the `Ad_Var` class to compute the value of the derivative for this function automatically, unlike the manual computational trace drawn out earlier. 

First, we create an instance of the `Ad_Var` object, with the value of $x = \dfrac{\pi}{6}$ assigned to the input variable, `val`.

In [4]:
a = np.pi / 6

x = Ad_Var(a)

The user should note that the `AutoDiff` package assumes that for a single input, the object being initialised will have a derivative value of 1 (stored as a Class attribute `self._ders`).

Next, we create `f`, which represents the full function. The `Ad_Var` object from the previous code can be used with dunder functions and additional functions within `Ad_Var` class to construct the full function being evaluated. 

In [5]:
f = (Ad_Var.sin(2*x))**2

As the functions are applied to the original `Ad_Var` object `x`, the `_val` and `_ders` attributes of the object are being updated with new values. The object `f`, representing the full function, will have its `_val` and `_ders` attributes containing the actual function and derivative values respectively.

To note: the user does not have the ability to manually set function and derivative values outside of instance initialization. Instead, the `_val` and `_ders` attributes will be made pseudoprivate.

The associated function value and derivative(s) of any `Ad_Var` instance may be retrieved through the `get_val` and `get_ders` functions as shown below:

In [6]:
print(f.get_val(), f.get_ders())

0.7499999999999999 1.7320508075688776


This almost returns us the values that we would have computed manually, except for the rounding error from floating point operations in Python. To verify this, we run the following assert statements on the function and derivative values computed using `numpy`, using $f = sin^2(2x)$ and $f' = 4sin(2x)cos(2x)$

In [7]:
val = np.pi / 6

assert(f.get_val()== (np.sin(2*val)**2))
assert(f.get_ders()== (4*np.sin(2*val)*np.cos(2*val)))

## Software Organization

### Directory structure
Our intended directory structure is as follows:
```
cs207-FinalProject/
                   README.md
                   requirements.txt 
                   docs/
                        milestone1.pdf
                        milestone2.ipynb
                   src/                   
                        AutoDiff.py
                   test/
                        test_autodiff.py
                       
```                 

### Modules

The primary module will be a single `AutoDiff.py` file. Contained within will be the definition for an `Ad_Var` class. Instances of this class, through interaction with other `Ad_Var` objects, will be able to compute the value of a function as well as the value of that function's derivative with respect to any input variable. At present, we envision that this module will be powerful enough to handle forward differentiation of any function comprised of the following elementary functions:

* Fundamental arithmetic operators (addition, subtraction, multiplication, and division)
* Logarithm (of any base)
* Negation
* Exponentiation ($e^x$ for an `Ad_Var` instance $x$)
* Power and root functions ($x^n$ for some real $n$)
* Trigonometric functions ($\sin(x)$, $\cos(x)$, $\tan(x)$)
* Inverse trigonometric functions ($\arcsin(x)$, $\arccos(x)$, $\arctan(x)$)

Depending on our eventual choice for the "additional" feature of this project, or future design decisions, there
may be additional modules added in the future that supplement or subdivide the functionality of `AutoDiff.py`.

Each instance of the `Ad_Var` class in the `AutoDiff` package represents the definition of a set of variables at a particular evaluation point. Through manipulations of these instances (either through fundamental arithmetic operations or built-in methods representing additional elementary functions described earlier), a user has the capability of representing any continuous differentiable function, be it scalar or vector. This was shown earlier via a code demo.


### Testing and Coverage
In the `test` folder, there will be a separate Python module `test_autodiff.py`, which will be the test-suite for `AutoDiff.py`. 

The test-suite will contain tests for the methods in the `Ad_Var` class, to ensure that the elementary functions return the desired output. Tests are run using pytest. The tests are linked to Travis CI and CodeCov, which will manage continuous integration and code coverage respectively.

### Installation and distribution of package
At this milestone, the recommended method to access the package is to download or clone the package's Github repo (https://github.com/the-differentiators/cs207-FinalProject.git). The user must ensure that the package's requirements (`numpy`) are installed, or install them manually.

By the final submission, we intend to distribute the package via PyPI. There will be no additional packaging framework included; we believe the scope of this project can be contained within a relatively simple directory structure with few functional python files and should not require additional overhead for users to install and use.

The package will be available on PyPI and the user will be able to install the package in the standard way using `pip`. We will also provide a `yml` file which will be used by the user to create the appropriate environment and ensure that dependencies like `numpy` have been installed. Then, to use the package, the user will only need to import our package which will implicitly import any other packages used by `AutoDiff`.

## Implementation Details

### Core Data Structures
* `numpy` arrays: 1-D `numpy` arrays will be used to keep the gradient vectors as the entire trace is evaluated. `numpy`
provides vectorized operations which will make the overloading of elementary functions much more efficient for
multivariate functions. If a vector function is provided, 2-D `numpy` arrays will be used to hold the Jacobian matrix.

### Class Implementation
* The `Ad_Var`  class will represent the variables that are used in the Automatic Differentiation process. In the case of a single input, the instance should be initialized with, `val`, a scalar value of that variable to be evaluated on when calculating both the function and derivative values (as shown in the demo above)

* In the case of multiple inputs, each input will be initialized as an `Ad_Var` object, with inputs `val`, a scalar value of that variable and `ders`, a `numpy` array representing the derivative of the input with regards to the other variables. An example is shown below:

In [8]:
x1 = Ad_Var(1, np.array([1, 0, 0]))
x2 = Ad_Var(2, np.array([0, 1, 0]))
x3 = Ad_Var(3, np.array([0, 0, 1]))

* Dunder methods such as "add" and "mul", and other elementary functions will be implemented under this class. More information on this is covered below in the *Class Methods* section. 

* As part of the class methods, we have included two static methods, `get_jacobian` and `get_values`, which respectively compute the Jacobian matrix and an array of function values for an array of `Ad_Var` objects. 

* In our implementation, we will also use the try-except method to catch unexpected input types: for example, if the user initializes the variable value of the `Ad_Var` instance with a value of type string, which is not a valid input type.

### Core Attributes
* `val`: float value, indicating the function value of the `Ad_Var` object evaluated at the given point
* `ders` (for single input): float value, indicating the derivative value of `Ad_Var` object evaluated at the given point
* `ders` (for multiple inputs): 1-D array of floats, representing the value of the derivatives of the multiple inputs evaluated at the given point 

* `val` and `ders` attributes will be made pseudoprivate to prevent users from manually setting function and derivative values outside of instance initialization

### External Dependencies
* `numpy` for implementation of the elementary functions (e.g. sin, sqrt, log and exp), by overloading `numpy` implementations for these functions
* `pytest` for testing
* TravisCI and CodeCov used to manage continuous integration and code coverage

### Class Methods

1. `__init__(self, val, ders=1)`:
    * Sets self._val to the argument `val`
    * Sets self._ders to the argument `ders`
    
    
2. `__eq__(self, other)`:
    * Returns True if self.val == other.val and self.ders == other.ders, returns False otherwise
    
    
3. `_repr__(self)`:
    * Returns a string representing the value of `self._val` (Value) and the value of `self._ders` (Gradient)
    
    
4. `get_val(self)`:
    * Returns the value of the attribute `self._val` 
    
    
5. `get_ders(self)`:
    * Returns the value of the attribute `self._ders` 


6. `__add__(self, other)` and `__radd__(self, other)`:
    * Other can be a float, int or AutoDiff object
    * Returns an `Ad_Var` object when calculating self + other or other + self


7. `__sub__(self, other)` and `__rsub__(self, other)`:
    * Other can be a float, int or AutoDiff object
    * Returns an `Ad_Var` object when calculating self - other or other - self


8. `__mul__(self, other)` and `__rmul__(self, other)`:
    * Other can be a float, int or AutoDiff object
    * Returns an `Ad_Var` object when calculating self * other or other * self

9. `__truediv__(self, other)` and `__rtruediv__(self, other)`:
    * Other can be a float, int or AutoDiff object
    * Returns an `Ad_Var` object when calculating self / other or other / self


10. `__pow__(self, r)`:
    * Other can be a float, int
    * Returns an `Ad_Var` object when calculating self ** other


11. `exp(self)`:
    * Returns an `Ad_Var` object with `self._val = np.exp(self._val)` and `self._deres = np.exp(self._val) * self._ders`


12. `log(self, logbase=np.e)`:
    * Optional argument for `logbase` (can be a float or int). By default, `logbase` is set to the exponential.
    * Returns an `Ad_Var` object with `self._val = np.log(self._val) / np.log(logbase)` and `self._ders = self._ders / (self._val * np.log(logbase))`


13. `sin(self)` and `cos(self)` and `tan(self)`:
    * Returns an `Ad_Var` object with `self._val` and `self._ders` updated accordingly based on the given trigonometric function
    
    
14. `arcsin(self)` and `arccos(self)` and `arctan(self)`:
    * Returns an `Ad_Var` object with `self._val` and `self._ders` updated accordingly based on the given inverse trigonometric function
    
    
15. `get_jacobian(functions_array, functions_dim, vars_dim)`:
    * Static method that returns the Jacobian matrix for a given array of `Ad_Var` objects 
    
    
16. `get_values(functions_array)`:
    * Static method that returns the an array of function values for a given array of `Ad_Var` objects

## Future Features

Our next steps will be to implement the reverse mode of Automatic Differentiation. The main challenge for this is to translate the conceptual framework of reverse mode into functioning code that builds on existing code. This might require creating a new class specifically for the reverse mode that replicates similar functions already in the existing `Ad_Var`. 

Building on our implementation of reverse mode of Auto Differentiation, a future feature that we plan to implement would be a feature that automatically chooses between the forward mode or reverse mode of Automatic Differentiation, based on what is optimal based on the number of parameters and functions. This will be presented to the user of the package to facilitate their decision-making.

In terms of the forward mode of Auto Differentiation, we will work on refining the `get_jacobian` method that would return the Jacobian matrix for an array of `Ad_Var` objects. This will allow the user to access the Jacobian via the `Ad_Var` class. We will also include doc-strings for the code, so ensure that the code is accessible to the user.

#### References

* [A Hitchhiker’s Guide to Automatic Differentiation](https://link.springer.com/article/10.1007/s11075-015-0067-6)
* Harvard CS207 2019 course materials